In [1]:
from azure.quantum import Workspace
workspace = Workspace (
    subscription_id = "83e0e071-c631-4700-bd13-7c6a2545ab1c",
    resource_group = "azurequantum",
    name = "jcrandall-q",
    location = "eastus"
)


# Shor's Algorithm
Given: $M$, the numbers to be factored:
* Step 1: Pick $n$ qubits so that $N = 2^n > M$.
    * $ M = 35 $
    * $ n = 11 $
    * $ N = 2048 $
* Step 2: Pick a random $ a < M $.
* Step 3: If $\text{gcd}(a,M) \neq 1$, we've factored $M$, so stop.
* Step 4: If not, use $a$ and $M$ in circuit